# Financial Parser

some description here...

## All imports from modules path

In [1]:
# import my personal modules
from modules import finviz_parser as fp
from modules import dividends_parser as dp
from modules import finviz_screener_parser as fsp

In [23]:
import yfinance as yf
import pandas as pd
import time
import os

## Finviz parsing part

In this part I parse info from finviz.com with beautiful soup and selenium modules.

* parsing finviz.com screener

* parsing yahoo finance

* parsing dividends info

* parsing elite.finviz.com 
    - quarterly reports
    - yearly reports  

* parsing CEO info

In [10]:
data = pd.read_excel('./data/Screener tests.xlsx')
top_data = data[data['Cap M$'] != 'No data on finviz'][:100]
tickers = list(top_data['Все'])

### finviz.com screener parsing

In [3]:
screener_df = fsp.parse_screener()
screener_df = fsp.market_cap_handler(screener_df)
screener_df

,Ticker,Company,Sector,Industry,Country,"Market Cap, M",P/E,Price,Change,Volume
0,SNDL,Sundial Growers Inc.,Healthcare,Drug Manufacturers - Specialty & Generic,Canada,27100,-,1.53,5.52%,"866,543,568"
1,PLTR,Palantir Technologies Inc.,Technology,Software - Infrastructure,USA,524200,-,29.00,15.22%,"316,135,227"
2,WATT,Energous Corporation,Technology,Scientific & Technical Instruments,USA,315,-,7.23,94.35%,"176,508,809"
3,ONTX,"Onconova Therapeutics, Inc.",Healthcare,Biotechnology,USA,338,-,1.63,-7.10%,"95,444,109"
4,CTRM,Castor Maritime Inc.,Industrials,Marine Shipping,Cyprus,615,-,1.21,0.00%,"88,635,647"
...,...,...,...,...,...,...,...,...,...,...
95,SAN,"Banco Santander, S.A.",Financial,Banks - Diversified,Spain,630200,-,3.64,3.70%,"16,771,842"
96,COTY,Coty Inc.,Consumer Defensive,Household & Personal Products,USA,60400,-,7.69,7.70%,"16,620,194"
97,TIGR,UP Fintech Holding Limited,Financial,Capital Markets,China,45800,672.69,34.98,11.33%,"16,499,222"
98,NLY,"Annaly Capital Management, Inc.",Real Estate,REIT - Mortgage,USA,116300,-,8.34,1.71%,"16,360,439"


### finance.yahoo.com parsing 

In [6]:
datas_dict = {}
all_data = pd.DataFrame(columns=['Ticker', 'Type', 'Value, $'])

In [7]:
for ticker in tickers:
    data = yf.download(ticker,'2020-01-01','2020-12-31')
    datas_dict[ticker] = data
    
    for i in range(5):
        new_data = pd.DataFrame(columns=['Ticker', 'Type', 'Value, $'], index=data.index)

        new_data['Ticker'] = ticker

        new_data['Type'] = data.columns[i]
        new_data['Value, $'] = data[data.columns[i]]
        all_data = pd.concat([all_data, new_data])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
all_data.index.name = 'Date'
all_data = all_data.sort_values(['Date', 'Ticker', 'Type'], ascending=[True, True, False])

In [9]:
all_data

,Ticker,Type,"Value, $"
Date,,,
2019-12-31,AAPL,Open,72.482498
2019-12-31,AAPL,Low,72.379997
2019-12-31,AAPL,High,73.419998
2019-12-31,AAPL,Close,73.412498
2019-12-31,AAPL,Adj Close,72.675339
...,...,...,...
2020-12-30,ZM,Open,358.420013
2020-12-30,ZM,Low,345.399994
2020-12-30,ZM,High,358.700012


In [11]:
all_data.to_csv('./data/quotes_2020.csv')

### dividends.com info parsing

In [14]:
df_dic = {}

for ticker in tickers:
    
    try:
        text = dp.get_text(dp.go_to_search_page(), ticker)
        div_dic = dp.reshape_div_dic(dp.make_div_dic(text))

        df_dic[ticker] = dp.make_df(div_dic)
    except:
        print(f'{ticker} - no dividends info')

AMZN - no dividends info
GOOG - no dividends info
GOOGL - no dividends info
TSLA - no dividends info
FB - no dividends info
BABA - no dividends info
BRK-B - no dividends info
BRK-A - no dividends info
PYPL - no dividends info
VZ - no dividends info
ADBE - no dividends info
NFLX - no dividends info
PDD - no dividends info
CRM - no dividends info
ORCL - no dividends info
TMUS - no dividends info
SHOP - no dividends info
JD - no dividends info
CHTR - no dividends info
BA - no dividends info
SE - no dividends info
ZM - no dividends info
AMD - no dividends info
ABNB - no dividends info
RTX - no dividends info


In [16]:
df_dic['PG']

,PAY DATE,DECLARED DATE,EX-DIVIDEND DATE,PAYOUT AMOUNT,QUALIFIED DIVIDEND?,PAYOUT TYPE,FREQUENCY,DAYS TAKEN FOR STOCK PRICE TO RECOVER
0,2020-11-16,2020-10-13,2020-10-22,0.7907,"Income, Qualified",Regular,Quarterly,16
1,2020-08-17,2020-07-14,2020-07-23,0.7907,"Income, Qualified",Regular,Quarterly,0
2,2020-05-15,2020-04-14,2020-04-23,0.7907,"Income, Qualified",Regular,Quarterly,0
3,2020-02-18,2020-01-14,2020-01-23,0.7907,"Income, Qualified",Regular,Quarterly,9
4,2019-11-15,2019-10-08,2019-10-17,0.7907,"Income, Qualified",Regular,Quarterly,2
...,...,...,...,...,...,...,...,...
107,1994-02-15,1994-01-11,1994-01-14,0.7907,Income,Regular,Quarterly,0
108,1993-11-15,1993-10-12,1993-10-18,0.7907,Income,Regular,Quarterly,7
109,1993-08-16,1993-07-15,1993-07-19,0.7907,Income,Regular,Quarterly,63
110,1993-05-14,1993-04-13,1993-04-19,0.7907,Income,Regular,Quarterly,6


In [18]:
os.mkdir('./data/dividends')

for key in df_dic.keys():
    df_dic[key].to_csv(f'./data/dividends/{key}_dividends_history.csv')

### elite.finviz.com parsing

In [19]:
email = 'Podolskiyant@gmail.com'
password = '9w3msoidofyi'

In [24]:
os.mkdir('./data/quarterly_reports')
os.mkdir('./data/yearly_reports')

In [25]:
wd = fp.login_on_finviz(email, password)

for ticker in tickers:    
    wd = fp.go_to_company_page(wd, ticker)
    
    try:
        yearly_table_list = fp.get_table_list(wd)
        yearly_df = fp.make_yearly_df(yearly_table_list, wd)
        yearly_df.to_excel(f'./data/yearly_reports/{ticker}_yearly_report.xlsx')
        print(f'{ticker} yearly - DONE!')
        time.sleep(2)
    except:
        print(f'{ticker} yearly - FAILED!')
        
    try:
        wd = fp.quarterly_table(wd)
        table_list = fp.get_table_list(wd)
        df = fp.make_df(table_list, wd)
        df.to_excel(f'./data/quarterly_reports/{ticker}_quarterly_report.xlsx')
        print(f'{ticker} quarterly - DONE!')
        time.sleep(3)
    except:
        print(f'{ticker} quarterly - FAILED!')
    
wd.close()

AAPL yearly - DONE!
AAPL quarterly - DONE!
MSFT yearly - DONE!
MSFT quarterly - DONE!
AMZN yearly - DONE!
AMZN quarterly - DONE!
GOOG yearly - DONE!
GOOG quarterly - DONE!
GOOGL yearly - DONE!
GOOGL quarterly - DONE!
TSLA yearly - DONE!
TSLA quarterly - DONE!
FB yearly - DONE!
FB quarterly - DONE!
BABA yearly - DONE!
BABA quarterly - DONE!
BRK-B yearly - DONE!
BRK-B quarterly - DONE!
BRK-A yearly - DONE!
BRK-A quarterly - DONE!
TSM yearly - DONE!
TSM quarterly - DONE!
V yearly - DONE!
V quarterly - DONE!
JPM yearly - DONE!
JPM quarterly - DONE!
JNJ yearly - DONE!
JNJ quarterly - DONE!
WMT yearly - DONE!
WMT quarterly - DONE!
PG yearly - DONE!
PG quarterly - DONE!
UNH yearly - DONE!
UNH quarterly - DONE!
NVDA yearly - DONE!
NVDA quarterly - DONE!
MA yearly - DONE!
MA quarterly - DONE!
DIS yearly - DONE!
DIS quarterly - DONE!
BAC yearly - DONE!
BAC quarterly - DONE!
HD yearly - DONE!
HD quarterly - DONE!
PYPL yearly - DONE!
PYPL quarterly - DONE!
TM yearly - DONE!
TM quarterly - DONE!
IN